In [1]:
import random
import pandas as pd

In [2]:
from collections import defaultdict

def deck_tracker():
    deck_1=[2, 3, 4, 5, 6, 7, 8, 9,10,11]
    prob_dict=defaultdict(int)
    for card in (deck_1):
        if card !=10:
            prob_dict[card]=4
        else:
            prob_dict[card]=16
    return prob_dict

# prob_dict=deck_tracker()

In [3]:
def get_total(deck):
    total=0    
    for key,value in prob_dict.items():
        total+=value
    return total



def cards_before(deck,number):
    total=1    
    for key,value in prob_dict.items():
#         print(key)
        if key!=number:
            total+=1
        else:
            return total


In [4]:
def calc_prob(card,deck,total_player):
    remaining=21-total_player
#     less_than=list(range(2,remaining+1))
    # prob_deck takes into account for all cards played and what is in the players and dealers hand
    total_prob=0
    for key,value in deck.items():
        if key<=remaining:
            single_prob=(value/get_total(deck))*100
            total_prob+=single_prob
    return total_prob

 


In [5]:
def deck_init():
    
    deck=[2, 3, 4, 5, 6, 7, 8, 9, 10, 10,10,10,11]*4
    random.shuffle(deck)
    tracker=deck_tracker()


    return deck,tracker

In [6]:
def record_init():
    record = defaultdict(list,{ key:[] for key in ('Player_Hand','Dealer_Hand','Player_Action','Player_Prob','Win','Loss') })
    return record
def update_record(record,phand,dhand,action,prob,win,loss):
    record['Player_Hand'].append(phand)
    record['Dealer_Hand'].append(dhand)
    record['Player_Action'].append(action)
    record['Player_Prob'].append(prob)
    record['Win'].append(win)
    record['Loss'].append(loss)
    return record
    
    
    

In [7]:
## COUNTING CARD

import random
from collections import defaultdict

loss=0
win=0
new_deck=False
deck,prob_dict=deck_init()
card_value=None
player_hand=[]
player_index=0
player_total=0
player_ace=0

percent_thresh=55
num_games=100
total_thresh=None
count=0
dealer_hand=[]
dealer_index=0
dealer_total=0
dealer_ace=0
d_stand=False
p_stand=False
record_sheet=record_init()



while (loss+win)<num_games:
    
    deck,prob_dict=deck_init()

    for card in range(len(deck)-1):
        temp_card=deck[card]
        prob_dict[temp_card]-=1
        
        if (loss+win)==num_games:
            break
        

        
    ##Initiate_Player


        if player_index<2:

            player_hand.append(temp_card)
            player_total=sum(player_hand)

            if player_index==1:
                player_prob=calc_prob(temp_card,prob_dict,player_total)
                if player_prob<percent_thresh:
                    p_stand=True

    
            if sum(player_hand)==21:
                    win+=1
                    record_sheet=update_record(record_sheet,player_hand,dealer_hand,'Start','Start',1,0)
                    dealer_index=0
                    player_index=0
                    player_total=0
                    dealer_total=0
                    player_hand=[]
                    dealer_hand=[]
                    d_stand=False
                    p_stand=False
                    continue        
            player_index+=1

            continue
        
        
        
        


        ##Initiate_Dealer
        if dealer_index<2:
            dealer_index+=1
            dealer_hand.append(temp_card)
            dealer_total=sum(dealer_hand)

            if dealer_total>16:
                d_stand=True

                if sum(dealer_hand)==21:
                    loss+=1
                    record_sheet=update_record(record_sheet,player_hand,dealer_hand,'Start','Start',0,1)
                    dealer_index=0
                    player_index=0
                    player_total=0
                    dealer_total=0
                    player_hand=[]
                    dealer_hand=[]
                    d_stand=False
                    p_stand=False
                    continue
            

            continue




            ### USE LOGIC FUNCTION
        if p_stand==False:
            player_prob=calc_prob(temp_card,prob_dict,player_total)
            
            if player_prob>=percent_thresh:

                player_hand.append(temp_card)
                player_total=sum(player_hand)


            if player_prob<percent_thresh:

                p_stand=True
                hit=True




            if player_total>21:
                loss+=1
                record_sheet=update_record(record_sheet,player_hand,dealer_hand,'Hit',player_prob,0,1)
                player_total=0
                dealer_total=0
                dealer_index=0
                player_index=0
                player_hand=[]
                dealer_hand=[]
                d_stand=False
                p_stand=False
                continue
            


    ## -------------------Dealers Have Own Constant Strategy      
        if d_stand==False:

            dealer_hand.append(temp_card)
            dealer_total=sum(dealer_hand)



            if dealer_total<16:
                pass

                
            if dealer_total>16:
                d_stand=True
                
                
            if dealer_total>21:
                win+=1
                record_sheet=update_record(record_sheet,player_hand,dealer_hand,'Stand',player_prob,1,0)
                dealer_index=0
                player_index=0
                player_total=0
                dealer_total=0
                player_hand=[]
                dealer_hand=[]
                d_stand=False
                p_stand=False

                continue


        if p_stand==False or d_stand==False:
            continue
            
        if p_stand==True and d_stand==True:


            if player_total>dealer_total:
                win+=1
                record_sheet=update_record(record_sheet,player_hand,dealer_hand,'Stand',player_prob,1,0)

            else:
                loss+=1
                record_sheet=update_record(record_sheet,player_hand,dealer_hand,'Stand',player_prob,0,1)

            dealer_index=0
            player_index=0
            player_total=0
            dealer_total=0
            player_hand=[]
            dealer_hand=[]
            d_stand=False
            p_stand=False
            continue



In [9]:

record_df=pd.DataFrame.from_dict(record_sheet)
record_df

,Player_Hand,Dealer_Hand,Player_Action,Player_Prob,Win,Loss
0,"[10, 2, 10]","[6, 5]",Hit,61.7021,0,1
1,"[7, 10]","[2, 11, 10]",Stand,24.4444,1,0
2,"[10, 9]","[11, 10]",Start,Start,0,1
3,"[9, 5]","[10, 7]",Stand,50,0,1
4,"[7, 10]","[10, 8]",Stand,32.2581,0,1
...,...,...,...,...,...,...
95,"[10, 3, 7]","[5, 6, 7]",Stand,0,1,0
96,"[9, 10]","[5, 11, 7]",Stand,12.5,1,0
97,"[8, 8]","[10, 2, 10]",Stand,42.1053,1,0
98,"[11, 8]","[6, 10, 5]",Stand,14.2857,0,1
